In [2]:
pip install autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 37.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=6c45aab6367c967a656eefc1f32520bd46be0a8ceeb0e4fc792d4ca8eb5a0ed2
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 47.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=ce4f90513a704977bb2c8006606ffee74a1f11df7b02b6b96bf408ba1dfba5a8
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install langid

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [69]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from autocorrect import Speller
from nltk.stem import PorterStemmer
from langdetect import detect
import langid
import contractions
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm

In [7]:
import nltk
# Set the data path explicitly
nltk.data.path.append('/kaggle/working/nltk_data')

# Download WordNet data
nltk.download('wordnet', download_dir='/kaggle/working/nltk_data')

[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data...


True

In [8]:
import zipfile
import os

zip_path = '/kaggle/working/nltk_data/corpora/wordnet.zip'
extract_path = '/kaggle/working/nltk_data/corpora'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [9]:
data= pd.read_csv(r'/kaggle/input/imdb-movie-ratings-sentiment-analysis/movie.csv')

In [10]:
data.head(10)

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
5,A terrible movie as everyone has said. What ma...,0
6,Finally watched this shocking movie last night...,1
7,I caught this film on AZN on cable. It sounded...,0
8,It may be the remake of 1987 Autumn's Tale aft...,1
9,My Super Ex Girlfriend turned out to be a plea...,1


In [11]:
X= data['text']
y= data['label']

# Text Data Proprocessing:

In [12]:
import re

def text_cleaning(text):
    # Step 1: Convert text to lowercase
    text = text.lower()

    # Step 2: Remove HTML tags
    text = re.sub(pattern=r'<.*?>', repl='', string=text)

    # Step 3: Remove URLs
    text = re.sub(pattern=r'http\S+|www\S+|ftp\S+', repl='', string=text)

    # Step 4: Remove 'rt'
    text = re.sub(pattern=r' rt', repl='', string=text)

    # Step 5: Remove words starting with '!'
    text = re.sub(pattern=r'!\S+', repl='', string=text)

    # Step 6: Remove mentions starting with '@'
    #text = re.sub(pattern=r'@\S+', repl='', string=text)

    # Step 7: Remove Unicode emojis
    text = re.sub(pattern=r'&#\d+;', repl='', string=text)
    
    text = re.sub(pattern=r'&amp', repl='', string=text)

    # Step 8: Remove '#' and '$'
    text = re.sub(pattern=r'#|\$', repl='', string=text)

    # Step 9: Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Step 10: Remove numeric digits
    text = re.sub(r'\d+', '', text)

    # Step 11: Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Step 12: Remove single characters
    text = re.sub(r'\b\w\b', '', text)

    # Step 13: Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [13]:
X= X.apply(text_cleaning)

In [14]:
print(type(X))


<class 'pandas.core.series.Series'>


In [15]:
X[:5]


0    grew up watching and loving the thunderbirds a...
1    when put this movie in my dvd player and sat d...
2    why do people who do not know what particular ...
3    even though have great interest in biblical mo...
4    im die hard dads army fan and nothing will eve...
Name: text, dtype: object

Stopwords Removal:

In [16]:
def stopwords_removal(text):
    words= word_tokenize(text)
    stopword= set(stopwords.words('english'))
    filtered_text= [word for word in words if word.lower() not in stopword]
    return ' '.join(filtered_text)

In [17]:
X= X.apply(stopwords_removal)
X[:5]

0    grew watching loving thunderbirds mates school...
1    put movie dvd player sat coke chips expectatio...
2    people know particular time past like feel nee...
3    even though great interest biblical movies bor...
4    im die hard dads army fan nothing ever change ...
Name: text, dtype: object

Stemming:

In [18]:
# def stemming(text):
#     words= word_tokenize(text)
#     stemmer= PorterStemmer()
#     stemmed_text= [stemmer.stem(word) for word in words]
#     return ' '.join(stemmed_text)

In [19]:
# X1= X.apply(stemming)
# X1[:5]

Lemmatization:

In [20]:
def lemmeaizing(text):
    words= word_tokenize(text)
    lemmatizer= WordNetLemmatizer()
    lemmatized_text= [lemmatizer.lemmatize(word,pos='v') for word in words]
    return ' '.join(lemmatized_text)

In [21]:
X2= X.apply(lemmeaizing)

Language Detection:

In [22]:
# def lang_detection(text):
#     return detect(text)

In [23]:
# X_new= X[:10]

In [24]:
# lang= X2.apply(lang_detection)
# lang

In [25]:
# def lang_detection(text):
#     return langid.classify(text)

In [26]:
# lang= X2.apply(lang_detection)
# print(lang)

Spelling Checker/ Correction:

In [27]:
# spell= Speller()
# def spell_correction(text):
#     return spell(text)

In [28]:
# wrong_text= 'hqllo, how are you?'
# corrected_text= spell_correction(wrong_text)
# print(f"Worng Text: {wrong_text}\nCorrected Text: {corrected_text}")

Chat Words Treatment:

In [29]:
# def chat_word_treatment(text):
#     return contractions.fix(text)

In [30]:
# # Example:

# text= 'u can do that asap'
# new_text= chat_word_treatment(text)
# print(f"text: {text}\nNew Text: {new_text}")

In [31]:
# X2= X2.apply(chat_word_treatment)

In [32]:
# X2[:5]

# Text Data Representation:

### One Hot Encoding Method: (OHE)

In [33]:
# from sklearn.preprocessing import OneHotEncoder

# reshaped_text= [[text] for text in X2]
# ohe= OneHotEncoder(sparse_output=True)
# transformed_ohe_text=ohe.fit_transform(reshaped_text).toarray()

In [34]:
# transformed_ohe_text.shape

In [35]:
# transformed_ohe_text[1]

### Bag Of Words: (BoW)

In [36]:
# from sklearn.feature_extraction.text import CountVectorizer

# cv= CountVectorizer()
# transformed_bow= cv.fit_transform(X2).toarray()


In [37]:
# feature_name=cv.get_feature_names_out()
# feature_name[:50]

In [38]:
# transformed_bow.shape

In [39]:
# transformed_bow[201,:300]

### N-Grams:

In [40]:
# # Taking Samples of 5000 Reviews....
# X2= X2[:5000]

In [41]:
# from sklearn.feature_extraction.text import CountVectorizer

# cv_ngm= CountVectorizer(ngram_range=(2,2))
# transformed_ngram= cv_ngm.fit_transform(X2).toarray()

In [42]:
# feature_name=cv_ngm.get_feature_names_out()
# feature_name[:50]

In [43]:
# transformed_ngram.shape

In [44]:
# transformed_ngram[504,:700]

### TF-IDF:

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf= TfidfVectorizer()
transformed_tf_idf= tf_idf.fit_transform(X2)

In [46]:
# Assuming tf_idf is your TF-IDF vectorizer
feature_names = tf_idf.get_feature_names_out()

# Display the first 10 items in the vocabulary
print(feature_names[:50])


['aa' 'aaa' 'aaaa' 'aaaaaaaaaaaahhhhhhhhhhhhhh' 'aaaaaaaargh' 'aaaaaaah'
 'aaaaaaahhhhhhggg' 'aaaaagh' 'aaaaah' 'aaaaargh'
 'aaaaarrrrrrgggggghhhhhh' 'aaaaaw' 'aaaahhhhhh' 'aaaahhhhhhh' 'aaaand'
 'aaaggghhhhhhh' 'aaah' 'aaahhhhhhh' 'aaahthe' 'aaargh' 'aachen' 'aada'
 'aadha' 'aag' 'aage' 'aagh' 'aaghh' 'aah' 'aahemy' 'aahhhh' 'aaila'
 'aailiyah' 'aaip' 'aaja' 'aajala' 'aak' 'aakash' 'aaker' 'aalcc'
 'aaliyah' 'aaliyahs' 'aalox' 'aames' 'aamess' 'aamilne' 'aamir' 'aamirs'
 'aamirso' 'aamr' 'aan']


In [47]:
transformed_tf_idf.shape

(40000, 171932)

In [48]:
transformed_tf_idf.toarray()[2,100:200]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [49]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(transformed_tf_idf,y,test_size=0.2)

In [50]:
from xgboost import XGBClassifier

model= XGBClassifier(n_estimators=500,n_jobs=-1)
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.867625

# Word2Vec:

In [51]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [52]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [53]:
tokenized_data= [word_tokenize(sentence) for sentence in X2]

In [54]:
model= Word2Vec(sentences= tokenized_data,vector_size=300,window=10,min_count=1,workers=4,sg=0)

In [55]:
# Most Similar Words:
model.wv.similar_by_word('good')

[('decent', 0.7154192924499512),
 ('nice', 0.633583664894104),
 ('great', 0.6295461058616638),
 ('fine', 0.5729900598526001),
 ('average', 0.5719606876373291),
 ('blevel', 0.5537705421447754),
 ('bad', 0.5504252910614014),
 ('cool', 0.5432404279708862),
 ('anywaysunfortunately', 0.5429602265357971),
 ('excellent', 0.5412538051605225)]

In [56]:
# Most Similar Words:

model.wv.similar_by_word('movie')

[('movies', 0.6383894681930542),
 ('moviei', 0.6012312769889832),
 ('honestly', 0.5576903820037842),
 ('film', 0.5490061044692993),
 ('itthe', 0.5268764495849609),
 ('moviethe', 0.5129658579826355),
 ('accentscarecrow', 0.5083079934120178),
 ('personally', 0.5025798678398132),
 ('shouldnt', 0.5009762048721313),
 ('talkingand', 0.495198130607605)]

In [57]:
# Most Similar Words:
model.wv.similar_by_word('review')

[('comment', 0.879792332649231),
 ('imdb', 0.8460783362388611),
 ('reviewers', 0.7639669179916382),
 ('users', 0.7576891183853149),
 ('reviewer', 0.7496821880340576),
 ('user', 0.7257369756698608),
 ('site', 0.6973509192466736),
 ('opinions', 0.6969476342201233),
 ('negative', 0.6936171054840088),
 ('website', 0.6824554800987244)]

In [58]:
# Most Similar Words:
model.wv.similar_by_word('actor')

[('role', 0.7374333143234253),
 ('actress', 0.7360354065895081),
 ('roles', 0.7194545865058899),
 ('spacey', 0.6770349740982056),
 ('foxx', 0.6691368222236633),
 ('performer', 0.6614894270896912),
 ('performance', 0.6612817645072937),
 ('actors', 0.6548554301261902),
 ('comedian', 0.6366715431213379),
 ('brosnan', 0.6359131932258606)]

In [59]:
# Similarity Between Words: (Most Similar then vector value is near to 1)
model.wv.similarity('good','decent')

0.71541923

In [60]:
# Similarity Between Words: (Most Similar then vector value is near to 0 and in -ve)
model.wv.similarity('good','monkey')

0.02275448

In [61]:
# Word Vector for a Word:
model.wv['movie']

array([-0.55167216,  0.6365483 , -1.056362  , -0.92899126, -1.1380368 ,
        0.15549438, -0.8158807 ,  0.89082515, -0.0956903 , -2.6667564 ,
        1.5182812 ,  1.0631413 , -0.7170675 , -0.24120957, -0.38583422,
       -0.03696334,  1.5005251 ,  0.04953648,  0.37851864,  1.5906037 ,
        0.6517655 , -0.820098  ,  0.1719952 ,  1.0497906 , -0.14250301,
        0.95447415,  0.6618943 ,  0.7120831 ,  0.9705653 , -0.08922379,
        0.19114567,  0.1695477 ,  1.7333668 ,  0.87892747,  0.42412305,
        1.6517241 ,  1.4363756 , -1.478833  , -0.16519202,  0.03923625,
       -0.7639232 , -0.02040668, -0.3291351 ,  0.1198689 ,  1.5504364 ,
       -0.07279108,  0.19155286,  0.11777131,  0.17479469, -1.4380264 ,
        0.80460656,  0.36354178,  0.30917847, -1.0429864 ,  1.0504702 ,
       -0.8269204 , -1.2461834 , -0.31380633, -0.16914316, -0.3540662 ,
        1.1199293 ,  0.3007899 ,  1.714321  ,  0.00333344, -0.4260804 ,
        1.5414982 ,  0.05053211,  0.74303585, -0.31486294,  1.45

In [62]:
analogy = model.wv.most_similar(positive=['drama', 'crime'], negative=['love'], topn=2)
analogy

[('procedural', 0.5993573069572449), ('murdersincubus', 0.5738452672958374)]

# Average Word2Vec:

In [64]:
tokenized_data[1]

['put',
 'movie',
 'dvd',
 'player',
 'sit',
 'coke',
 'chip',
 'expectations',
 'hop',
 'movie',
 'would',
 'contain',
 'strongpoints',
 'first',
 'movie',
 'awsome',
 'animation',
 'good',
 'flow',
 'story',
 'excellent',
 'voice',
 'cast',
 'funny',
 'comedy',
 'kickass',
 'soundtrack',
 'disappointment',
 'find',
 'atlantis',
 'milos',
 'return',
 'read',
 'review',
 'first',
 'might',
 'let',
 'follow',
 'paragraph',
 'direct',
 'see',
 'first',
 'movie',
 'enjoy',
 'primarily',
 'point',
 'mentionedwhen',
 'first',
 'scene',
 'appear',
 'shock',
 'pick',
 'atlantis',
 'milos',
 'return',
 'displaycase',
 'local',
 'videoshop',
 'whatever',
 'expectations',
 'music',
 'feel',
 'bad',
 'imitation',
 'first',
 'movie',
 'voice',
 'cast',
 'replace',
 'fit',
 'one',
 'exception',
 'character',
 'like',
 'voice',
 'sweet',
 'actual',
 'draw',
 'isnt',
 'bad',
 'animation',
 'particular',
 'sad',
 'sight',
 'storyline',
 'also',
 'pretty',
 'weak',
 'like',
 'three',
 'episodes',
 'sch

In [71]:
def avg_word2vec(doc, model):
    word_vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]

    if not word_vectors:
        # If there are no valid word vectors, return a default vector (zeros)
        return np.zeros(model.vector_size)

    # If there are valid word vectors, calculate the mean
    return np.mean(word_vectors, axis=0)

In [73]:
X=[]

for i in tqdm(range(len(tokenized_data))):
    X.append(avg_word2vec(tokenized_data[i],model))

100%|██████████| 40000/40000 [11:33<00:00, 57.70it/s] 


In [81]:
X= np.array(X)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [83]:
X.shape

(40000, 300)

In [92]:
X_train.shape

(32000, 300)

In [93]:
X_test.shape

(8000, 300)

In [106]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB,CategoricalNB,MultinomialNB
from xgboost import XGBClassifier


In [97]:
model= LogisticRegression(max_iter=1000)
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.864

In [99]:
model= SVC()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.8705

In [100]:
model= DecisionTreeClassifier()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.737

In [101]:
model= RandomForestClassifier()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.842625

In [102]:
model= GaussianNB()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.77625

In [107]:
model= XGBClassifier()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.854875